In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence_transformers
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=c8d241c70164cf9c3a1887b09a07b38a92abfe1f245468609342d3fb354d52bb
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from tqdm import tqdm
from operator import itemgetter

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def load_data(data_file):
  # read csv file
  df = pd.read_csv(data_file)

  # replace nan(no value) comment with ""(empty string)
  df.fillna("", inplace=True)

  comments = df['comments'].tolist()
  genders = df['labels'].tolist()

  genders = [0 if gender == "Male" else 1 for gender in genders]

  return comments, genders

In [ ]:
comments, genders = load_data('/content/drive/MyDrive/Datasets/Bert Modal On One Comment Full Data/cleaned_dataset.csv')

# df = pd.DataFrame(list(zip(comments, genders)),
#                columns =['comments', 'genders'])

# print(df)

comments = np.array(comments)
genders = np.array(genders)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(comments, genders,
                                                    test_size=0.2,
                                                    stratify=genders,
                                                    random_state=42)
print(len(X_train),len( X_test), len(y_train),len( y_test))

152083 38021 152083 38021


In [ ]:
# Model

top_words = 300000
max_comment_length = 300
embedding_vecor_length = 64

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_comment_length)

In [ ]:
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_comment_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 64)           19200064  
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 19266165 (73.49 MB)
Trainable params: 19266165 (73.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train, epochs=10, batch_size=64)

Epoch 1/10
2377/2377 [==============================] - 99s 38ms/step - loss: 0.5430 - accuracy: 0.7055
Epoch 2/10
2377/2377 [==============================] - 46s 19ms/step - loss: 0.4261 - accuracy: 0.7872
Epoch 3/10
2377/2377 [==============================] - 46s 19ms/step - loss: 0.3574 - accuracy: 0.8209
Epoch 4/10
2377/2377 [==============================] - 46s 19ms/step - loss: 0.3063 - accuracy: 0.8448
Epoch 5/10
2377/2377 [==============================] - 46s 19ms/step - loss: 0.2691 - accuracy: 0.8625
Epoch 6/10
2377/2377 [==============================] - 44s 19ms/step - loss: 0.2427 - accuracy: 0.8748
Epoch 7/10
2377/2377 [==============================] - 45s 19ms/step - loss: 0.2222 - accuracy: 0.8850
Epoch 8/10
2377/2377 [==============================] - 44s 19ms/step - loss: 0.2064 - accuracy: 0.8934
Epoch 9/10
2377/2377 [==============================] - 45s 19ms/step - loss: 0.1929 - accuracy: 0.8992
Epoch 10/10
2377/2377 [==============================] - 46s 19m

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(list_tokenized_test, maxlen=max_comment_length)
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)


1189/1189 [==============================] - 7s 5ms/step
Accuracy of the model :  0.7607111859235686
F1-score:  0.7428781370110784
Confusion matrix:


array([[15780,  4797],
       [ 4301, 13143]])

In [ ]:
def load_data_for_replaced_dataset(data_file):
  # read csv file
  df = pd.read_csv(data_file)

  # replace nan(no value) comment with ""(empty string)
  df.fillna("", inplace=True)

  comments = df['comment'].tolist()
  genders = df['user_gender'].tolist()

  genders = [0 if gender == "Male" else 1 for gender in genders]

  return comments, genders

In [ ]:
replaced_comments, replaced_genders = load_data_for_replaced_dataset('/content/drive/MyDrive/Datasets/RNN/combined_replaced_dataset_one_comment_rnn_with_changed.csv')

replaced_comments = np.array(replaced_comments)
replaced_genders = np.array(replaced_genders)

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(replaced_comments)
changed_replaced_comments = pad_sequences(list_tokenized_test, maxlen=max_comment_length)
prediction = model.predict(changed_replaced_comments)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, replaced_genders))
print('F1-score: ', f1_score(y_pred, replaced_genders))
print('Confusion matrix:')
confusion_matrix(replaced_genders,y_pred)


5941/5941 [==============================] - 34s 6ms/step
Accuracy of the model :  0.6171516643521441
F1-score:  0.6307238788173991
Confusion matrix:


array([[55168, 31874],
       [40907, 62155]])